In [1]:
# Вспомогательные функции для анализа и визуализации

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """Комплексная оценка модели с расчетом всех метрик."""
    from sklearn.model_selection import cross_val_score
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    # Обучение и предсказание
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    # OOB Score для ансамблевых методов
    oob_score = getattr(model, 'oob_score_', None)
    
    return {
        'Модель': model_name,
        'Test Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted'),
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'OOB Score': oob_score
    }

def create_comparison_table(results_df):
    """Создает интерактивную сводную таблицу результатов."""
    import plotly.graph_objects as go
    
    fig_summary = go.Figure(data=[go.Table(
        header=dict(values=list(results_df.columns),
                    fill_color='lightblue',
                    align='center',
                    font=dict(size=12, color='black')),
        cells=dict(values=[results_df[col] for col in results_df.columns],
                   fill_color='white',
                   align='center',
                   format=[None, '.4f', '.4f', '.4f', '.4f', '.4f', '.4f', '.4f'],
                   font=dict(size=11)))
    ])
    
    fig_summary.update_layout(
        title='Сводная таблица результатов всех моделей',
        height=400
    )
    
    return fig_summary

def create_metrics_chart(results_df):
    """Создает диаграмму сравнения основных метрик."""
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Основные метрики', 'Cross-Validation'],
        specs=[[{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Основные метрики
    metrics = ['Test Accuracy', 'Precision', 'Recall', 'F1-Score']
    colors = [COLOR_PALETTE['primary'], COLOR_PALETTE['secondary'], 
              COLOR_PALETTE['success'], COLOR_PALETTE['danger']]
    
    for i, metric in enumerate(metrics):
        fig.add_trace(
            go.Bar(x=results_df['Модель'], y=results_df[metric],
                   name=metric, marker_color=colors[i],
                   text=results_df[metric].round(3),
                   textposition='outside'),
            row=1, col=1
        )
    
    # Cross-Validation
    fig.add_trace(
        go.Bar(x=results_df['Модель'], y=results_df['CV Mean'],
               error_y=dict(type='data', array=results_df['CV Std']),
               name='CV Score ± Std', marker_color=COLOR_PALETTE['info'],
               text=results_df['CV Mean'].round(3),
               textposition='outside'),
        row=1, col=2
    )
    
    fig.update_layout(height=500, showlegend=True)
    return fig

def analyze_feature_importance(model, feature_names, top_n=10):
    """Анализирует важность признаков для модели."""
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'estimators_'):
        importances = np.mean([est.feature_importances_ for est in model.estimators_], axis=0)
    else:
        return None, None
    
    top_indices = np.argsort(importances)[-top_n:][::-1]
    top_names = [feature_names[i] for i in top_indices]
    top_values = importances[top_indices]
    
    return top_names, top_values

def create_importance_chart(model, feature_names, title="Важность признаков"):
    """Создает диаграмму важности признаков."""
    import plotly.graph_objects as go
    
    names, values = analyze_feature_importance(model, feature_names)
    if names is None:
        return None
    
    fig = go.Figure(data=[
        go.Bar(y=names, x=values, orientation='h',
               marker_color=COLOR_PALETTE['primary'],
               text=[f'{v:.3f}' for v in values],
               textposition='auto')
    ])
    
    fig.update_layout(
        title=title,
        xaxis_title='Важность',
        yaxis_title='Признаки',
        height=400
    )
    
    return fig

def print_model_summary(results_df):
    """Выводит краткое резюме результатов."""
    print("КРАТКИЙ АНАЛИЗ РЕЗУЛЬТАТОВ")
    print("=" * 40)
    
    best_model = results_df.loc[results_df['Test Accuracy'].idxmax()]
    print(f"Лучшая модель: {best_model['Модель']} (Accuracy: {best_model['Test Accuracy']:.4f})")
    
    most_stable = results_df.loc[results_df['CV Std'].idxmin()]
    print(f"Самая стабильная: {most_stable['Модель']} (CV Std: {most_stable['CV Std']:.4f})")
    
    # Анализ улучшений ансамблей
    ensemble_models = results_df[results_df['Модель'].isin(['Bagging', 'Random Forest'])]
    simple_dt = results_df[results_df['Модель'] == 'Decision Tree']['Test Accuracy'].iloc[0]
    
    print(f"\nПРЕИМУЩЕСТВА АНСАМБЛЕВЫХ МЕТОДОВ:")
    for _, model in ensemble_models.iterrows():
        improvement = ((model['Test Accuracy'] - simple_dt) / simple_dt) * 100
        print(f"   {model['Модель']}: +{improvement:.1f}% улучшение accuracy")

print("Вспомогательные функции загружены!")

Вспомогательные функции загружены!


# Intro to ML, Week 4: Decision Trees, Bagging и Random Forest

## Цели лабораторной работы

В рамках данной лабораторной работы мы изучим:
1. **Decision Trees** - основные принципы работы, математические основы, критерии разбиения
2. **Bagging** - метод Bootstrap Aggregating для уменьшения дисперсии моделей  
3. **Random Forest** - комбинация Bagging и случайного выбора признаков
4. **Сравнение методов** - анализ производительности одиночного дерева против ансамблевых подходов

## Структура работы

1. Теоретические основы каждого метода с математическими формулировками
2. Практическая реализация алгоритмов с подробными комментариями
3. Пошаговые примеры расчетов на небольших данных
4. Интерактивные элементы для экспериментирования  
5. Сравнительный анализ методов
6. Домашнее задание

---

## 1. Импорт библиотек и подготовка данных

Начнем с импорта всех необходимых библиотек для нашей работы:

In [2]:
# Импорт основных библиотек
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown
import warnings
warnings.filterwarnings('ignore')

# Настройка для воспроизводимости результатов  
np.random.seed(42)

# Единая цветовая палитра для всех визуализаций
COLOR_PALETTE = {
    'primary': '#2E86AB',      # Синий
    'secondary': '#A23B72',    # Розовый  
    'accent': '#F18F01',       # Оранжевый
    'success': '#32A251',      # Зеленый
    'danger': '#C73E1D',       # Красный
    'info': '#592E83',         # Фиолетовый
    'warning': '#FF7F0E'       # Оранжево-желтый
}

In [3]:
# Загрузка датасета Breast Cancer Wisconsin
def load_and_prepare_data():
    """
    Загружает и подготавливает датасет Breast Cancer Wisconsin для демонстрации 
    работы алгоритмов машинного обучения.
    
    Returns
    -------
    X : np.ndarray
        Матрица признаков (30 признаков)
    y : np.ndarray
        Вектор целевых значений (0 - злокачественная, 1 - доброкачественная)
    feature_names : list
        Названия признаков
    target_names : list
        Названия классов
    """
    # Загружаем данные
    data = load_breast_cancer()
    X, y = data.data, data.target
    feature_names = list(data.feature_names)
    target_names = list(data.target_names)
    
    return X, y, feature_names, target_names

# Загружаем данные
X, y, feature_names, target_names = load_and_prepare_data()

print(f"=== ИНФОРМАЦИЯ О ДАТАСЕТЕ BREAST CANCER WISCONSIN ===")
print(f"Размер датасета: {X.shape}")
print(f"Количество признаков: {X.shape[1]}")
print(f"Количество образцов: {X.shape[0]}")
print(f"Количество классов: {len(np.unique(y))}")
print(f"Классы: {target_names}")
print(f"Распределение классов: {dict(zip(target_names, np.bincount(y)))}")

# Первые 10 признаков для понимания данных
print(f"\nПервые 10 признаков:")
for i, name in enumerate(feature_names[:10]):
    print(f"{i+1:2d}. {name}")

print(f"\nСтатистика по признакам:")
print(f"Минимальные значения: {X.min(axis=0)[:5].round(2)}")
print(f"Максимальные значения: {X.max(axis=0)[:5].round(2)}")
print(f"Средние значения: {X.mean(axis=0)[:5].round(2)}")

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nРазмер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
print(f"Распределение в обучающей выборке: {dict(zip(target_names, np.bincount(y_train)))}")
print(f"Распределение в тестовой выборке: {dict(zip(target_names, np.bincount(y_test)))}")

=== ИНФОРМАЦИЯ О ДАТАСЕТЕ BREAST CANCER WISCONSIN ===
Размер датасета: (569, 30)
Количество признаков: 30
Количество образцов: 569
Количество классов: 2
Классы: ['malignant', 'benign']
Распределение классов: {'malignant': 212, 'benign': 357}

Первые 10 признаков:
 1. mean radius
 2. mean texture
 3. mean perimeter
 4. mean area
 5. mean smoothness
 6. mean compactness
 7. mean concavity
 8. mean concave points
 9. mean symmetry
10. mean fractal dimension

Статистика по признакам:
Минимальные значения: [6.980e+00 9.710e+00 4.379e+01 1.435e+02 5.000e-02]
Максимальные значения: [2.811e+01 3.928e+01 1.885e+02 2.501e+03 1.600e-01]
Средние значения: [1.4130e+01 1.9290e+01 9.1970e+01 6.5489e+02 1.0000e-01]

Размер обучающей выборки: (398, 30)
Размер тестовой выборки: (171, 30)
Распределение в обучающей выборке: {'malignant': 148, 'benign': 250}
Распределение в тестовой выборке: {'malignant': 64, 'benign': 107}


## 2. Теория Decision Trees

### 2.1 Математические основы

**Decision Tree** (дерево решений) — это алгоритм машинного обучения, который строит модель предсказания в виде древовидной структуры, где:
- Каждый внутренний узел представляет тест на значение признака
- Каждая ветвь представляет результат теста  
- Каждый листовой узел представляет метку класса

### 2.2 Формальное определение

Пусть у нас есть обучающая выборка $D = \{(x_i, y_i)\}_{i=1}^N$, где $x_i \in \mathbb{R}^d$ — вектор признаков, $y_i \in \{1, 2, ..., K\}$ — метка класса.

Дерево решений $T$ можно представить как функцию:
$$T: \mathbb{R}^d \rightarrow \{1, 2, ..., K\}$$

### 2.3 Критерии разбиения

Для построения дерева нужно найти оптимальные разбиения. Основные критерии:

#### 2.3.1 Gini Impurity (Индекс Джини)

Для узла $t$ с множеством объектов $S_t$:

$$Gini(t) = 1 - \sum_{k=1}^{K} p_k^2(t)$$

где $p_k(t) = \frac{|S_t^k|}{|S_t|}$ — доля объектов класса $k$ в узле $t$.

**Интуиция**: Gini измеряет "нечистоту" узла. Если все объекты в узле принадлежат одному классу, то $Gini = 0$. Максимальное значение достигается при равномерном распределении классов.

#### 2.3.2 Entropy (Энтропия)

$$Entropy(t) = -\sum_{k=1}^{K} p_k(t) \log_2 p_k(t)$$

**Интуиция**: Энтропия измеряет количество информации, необходимое для описания распределения классов в узле.

#### 2.3.3 Information Gain (Информационный выигрыш)

$$IG(S, A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} Entropy(S_v)$$

где $S_v$ — подмножество $S$, для которого атрибут $A$ имеет значение $v$.

### 2.4 Алгоритм построения дерева

1. **Начать с корневого узла**, содержащего всю обучающую выборку
2. **Для каждого узла**:
   - Если все объекты принадлежат одному классу → создать лист
   - Если достигнуто условие остановки → создать лист с majority class
   - Иначе: найти лучшее разбиение по критерию качества
3. **Рекурсивно применить** алгоритм к дочерним узлам

### 2.5 Плюсы и минусы Decision Trees

#### Преимущества:
- **Интерпретируемость**: Легко понять логику принятия решений
- **Работа с категориальными данными**: Не требует предобработки  
- **Нелинейные зависимости**: Может моделировать сложные границы
- **Выбор признаков**: Автоматически определяет важные признаки
- **Работа с пропусками**: Может обрабатывать missing values

#### Недостатки:
- **Переобучение**: Склонность к overfitting, особенно на глубоких деревьях
- **Неустойчивость**: Малые изменения в данных могут сильно изменить дерево
- **Смещение**: Предпочтение признаков с большим количеством значений
- **Сложность**: Не может эффективно представить линейные зависимости

## 3. Пошаговый пример построения Decision Tree

Рассмотрим построение дерева на небольшом примере, который можно прорешать вручную.

### 3.1 Исходные данные

Представим, что у нас есть следующий датасет для предсказания "Играть в теннис":

| Температура | Влажность | Ветер   | Играть |
|-------------|-----------|---------|--------|
| Высокая     | Высокая   | Слабый  | Нет    |
| Высокая     | Низкая    | Сильный | Нет    |
| Средняя     | Высокая   | Слабый  | Да     |
| Низкая      | Высокая   | Слабый  | Да     |
| Низкая      | Низкая    | Слабый  | Да     |
| Низкая      | Низкая    | Сильный | Нет    |
| Средняя     | Низкая    | Сильный | Да     |
| Высокая     | Высокая   | Слабый  | Нет    |

**Распределение целевого класса:** Да = 4, Нет = 4

### 3.2 Шаг 1: Вычисление начальной энтропии

Для корневого узла с 8 примерами (4 "Да", 4 "Нет"):

$$p(\text{Да}) = \frac{4}{8} = 0.5$$
$$p(\text{Нет}) = \frac{4}{8} = 0.5$$

$$\text{Entropy}(\text{корень}) = -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = -0.5 \cdot (-1) - 0.5 \cdot (-1) = 1.0$$

$$\text{Gini}(\text{корень}) = 1 - (0.5)^2 - (0.5)^2 = 1 - 0.25 - 0.25 = 0.5$$

### 3.3 Шаг 2: Анализ разбиения по признаку "Влажность"

Рассмотрим разбиение по признаку "Влажность":

**Влажность = Высокая:** примеры 1, 3, 4, 8
- Да = 2 (примеры 3, 4)
- Нет = 2 (примеры 1, 8)
- $p(\text{Да}) = \frac{2}{4} = 0.5$, $p(\text{Нет}) = \frac{2}{4} = 0.5$
- $\text{Entropy} = -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 1.0$

**Влажность = Низкая:** примеры 2, 5, 6, 7
- Да = 2 (примеры 5, 7)
- Нет = 2 (примеры 2, 6)
- $p(\text{Да}) = \frac{2}{4} = 0.5$, $p(\text{Нет}) = \frac{2}{4} = 0.5$
- $\text{Entropy} = 1.0$

**Information Gain для "Влажность":**
$$IG = 1.0 - \frac{4}{8} \cdot 1.0 - \frac{4}{8} \cdot 1.0 = 1.0 - 0.5 - 0.5 = 0.0$$

### 3.4 Шаг 3: Анализ разбиения по признаку "Температура"

**Температура = Высокая:** примеры 1, 2, 8
- Да = 0
- Нет = 3
- $\text{Entropy} = 0$ (чистый узел)

**Температура ≠ Высокая:** примеры 3, 4, 5, 6, 7
- Да = 4 (примеры 3, 4, 5, 7)
- Нет = 1 (пример 6)
- $p(\text{Да}) = \frac{4}{5} = 0.8$, $p(\text{Нет}) = \frac{1}{5} = 0.2$
- $\text{Entropy} = -0.8 \log_2(0.8) - 0.2 \log_2(0.2) = -0.8 \cdot (-0.322) - 0.2 \cdot (-2.322) = 0.258 + 0.464 = 0.722$

**Information Gain для "Температура":**
$$IG = 1.0 - \frac{3}{8} \cdot 0 - \frac{5}{8} \cdot 0.722 = 1.0 - 0 - 0.451 = 0.549$$

### 3.5 Вывод

Поскольку $IG(\text{Температура}) = 0.549 > IG(\text{Влажность}) = 0.0$, лучшим разбиением для корневого узла является **"Температура = Высокая"**.

Получившееся дерево:
```
                 Температура = Высокая?
                /                    \
              Да                      Нет
             /                         \
        Играть = НЕТ              [Да=4, Нет=1]
                                      |
                                 Дальнейшие
                                 разбиения...
```

In [4]:
# Функции для расчета критериев разбиения
def calculate_gini(y):
    """
    Вычисляет индекс Джини для вектора меток классов.
    
    Parameters
    ----------
    y : array-like
        Вектор меток классов
        
    Returns
    -------
    float
        Значение индекса Джини
    """
    if len(y) == 0:
        return 0
    
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    gini = 1 - np.sum(probabilities ** 2)
    return gini

In [5]:
def calculate_entropy(y):
    """
    Вычисляет энтропию для вектора меток классов.
    
    Parameters
    ----------
    y : array-like
        Вектор меток классов
        
    Returns
    -------
    float
        Значение энтропии
    """
    if len(y) == 0:
        return 0
    
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    # Избегаем log(0)
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))
    return entropy

In [6]:
def calculate_information_gain(parent_y, left_y, right_y, criterion='entropy'):
    """
    Вычисляет информационный выигрыш от разбиения.
    
    Parameters
    ----------
    parent_y : array-like
        Метки класса в родительском узле
    left_y : array-like  
        Метки класса в левом дочернем узле
    right_y : array-like
        Метки класса в правом дочернем узле
    criterion : str
        Критерий измерения ('entropy' или 'gini')
        
    Returns
    -------
    float
        Информационный выигрыш
    """
    n_parent = len(parent_y)
    n_left = len(left_y)
    n_right = len(right_y)
    
    if criterion == 'entropy':
        parent_impurity = calculate_entropy(parent_y)
        weighted_impurity = (n_left/n_parent) * calculate_entropy(left_y) + (n_right/n_parent) * calculate_entropy(right_y)
    else:  # gini
        parent_impurity = calculate_gini(parent_y)
        weighted_impurity = (n_left/n_parent) * calculate_gini(left_y) + (n_right/n_parent) * calculate_gini(right_y)
    
    return parent_impurity - weighted_impurity

## 4. Практическая реализация Decision Tree

In [7]:
# Создание и обучение Decision Tree для датасета Breast Cancer
dt_classifier = DecisionTreeClassifier(
    criterion='gini',           # Критерий разбиения
    max_depth=6,               # Оптимальная глубина для Breast Cancer
    min_samples_split=10,       # Минимальное количество образцов для разбиения
    min_samples_leaf=5,         # Минимальное количество образцов в листе
    max_features='sqrt',        # Случайный выбор признаков
    random_state=42
)

# Обучение модели
dt_classifier.fit(X_train, y_train)

print("=== НАСТРОЙКИ DECISION ===")
print(f"Максимальная глубина: {dt_classifier.max_depth}")
print(f"Количество признаков для рассмотрения: {dt_classifier.max_features}")
print(f"Минимум образцов для разбиения: {dt_classifier.min_samples_split}")
print(f"Минимум образцов в листе: {dt_classifier.min_samples_leaf}")
print(f"Итоговая глубина обученного дерева: {dt_classifier.get_depth()}")
print(f"Количество листьев: {dt_classifier.get_n_leaves()}")

=== НАСТРОЙКИ DECISION ===
Максимальная глубина: 6
Количество признаков для рассмотрения: sqrt
Минимум образцов для разбиения: 10
Минимум образцов в листе: 5
Итоговая глубина обученного дерева: 6
Количество листьев: 15


In [8]:
# Предсказания и оценка качества
y_pred_dt = dt_classifier.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')

print("=== РЕЗУЛЬТАТЫ DECISION TREE ===")
print(f"Accuracy:  {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall:    {recall_dt:.4f}")
print(f"F1-Score:  {f1_dt:.4f}")

# Анализ модели
print(f"\n=== АНАЛИЗ МОДЕЛИ ===")
print(f"Количество классов: {len(target_names)}")
print(f"Глубина обученного дерева: {dt_classifier.get_depth()}")
print(f"Количество листьев: {dt_classifier.get_n_leaves()}")

# Важность признаков
feature_importance = dt_classifier.feature_importances_
top_features = np.argsort(feature_importance)[-5:][::-1]
print(f"\nТоп-5 важных признаков:")
for i, feat_idx in enumerate(top_features):
    print(f"{i+1}. {feature_names[feat_idx]}: {feature_importance[feat_idx]:.4f}")

# Детальный отчет по классификации
print(f"\n=== ДЕТАЛЬНЫЙ ОТЧЕТ ===")
print(classification_report(y_test, y_pred_dt, target_names=target_names))

# Визуализация результатов с единой цветовой палитрой
fig_dt_results = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Распределение классов', 'Важность признаков (топ-10)', 
                   'Метрики качества', 'Точность по классам'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# График 1: Распределение классов
train_counts = np.bincount(y_train)
test_counts = np.bincount(y_test)

fig_dt_results.add_trace(
    go.Bar(x=target_names, 
           y=train_counts,
           name='Обучающая выборка',
           marker_color=COLOR_PALETTE['primary'],
           text=train_counts,
           textposition='auto'),
    row=1, col=1
)

fig_dt_results.add_trace(
    go.Bar(x=target_names, 
           y=test_counts,
           name='Тестовая выборка',
           marker_color=COLOR_PALETTE['secondary'],
           text=test_counts,
           textposition='auto'),
    row=1, col=1
)

# График 2: Важность признаков (топ-10)
top_10_indices = np.argsort(feature_importance)[-10:]
top_10_names = [feature_names[i][:20] + '...' if len(feature_names[i]) > 20 else feature_names[i] 
                for i in top_10_indices]
top_10_importance = feature_importance[top_10_indices]

fig_dt_results.add_trace(
    go.Bar(y=top_10_names, 
           x=top_10_importance,
           orientation='h',
           marker_color=COLOR_PALETTE['accent'],
           text=[f'{val:.3f}' for val in top_10_importance],
           textposition='auto',
           showlegend=False),
    row=1, col=2
)

# График 3: Основные метрики
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
values = [accuracy_dt, precision_dt, recall_dt, f1_dt]
colors = [COLOR_PALETTE['primary'], COLOR_PALETTE['secondary'], 
          COLOR_PALETTE['accent'], COLOR_PALETTE['success']]

fig_dt_results.add_trace(
    go.Bar(x=metrics, y=values,
           text=[f'{v:.3f}' for v in values],
           textposition='auto',
           marker_color=colors,
           showlegend=False),
    row=2, col=1
)

# График 4: Точность по классам
report = classification_report(y_test, y_pred_dt, target_names=target_names, output_dict=True)
class_f1_scores = [report[target_names[0]]['f1-score'], report[target_names[1]]['f1-score']]

fig_dt_results.add_trace(
    go.Bar(x=target_names, 
           y=class_f1_scores,
           marker_color=[COLOR_PALETTE['success'], COLOR_PALETTE['info']],
           text=[f'{val:.3f}' for val in class_f1_scores],
           textposition='auto',
           showlegend=False),
    row=2, col=2
)

fig_dt_results.update_layout(
    title='Детальный анализ Decision Tree',
    height=800,
    showlegend=True,
    font=dict(size=10),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_dt_results.show()

=== РЕЗУЛЬТАТЫ DECISION TREE ===
Accuracy:  0.9240
Precision: 0.9252
Recall:    0.9240
F1-Score:  0.9243

=== АНАЛИЗ МОДЕЛИ ===
Количество классов: 2
Глубина обученного дерева: 6
Количество листьев: 15

Топ-5 важных признаков:
1. worst radius: 0.7981
2. worst symmetry: 0.0688
3. worst concave points: 0.0357
4. mean concavity: 0.0181
5. mean texture: 0.0160

=== ДЕТАЛЬНЫЙ ОТЧЕТ ===
              precision    recall  f1-score   support

   malignant       0.88      0.92      0.90        64
      benign       0.95      0.93      0.94       107

    accuracy                           0.92       171
   macro avg       0.92      0.92      0.92       171
weighted avg       0.93      0.92      0.92       171



In [9]:
# Функция для анализа влияния гиперпараметров
def analyze_hyperparameters(max_depth=5, min_samples_split=20, min_samples_leaf=10):
    """
    Анализирует влияние гиперпараметров на Decision Tree.
    
    Parameters
    ----------
    max_depth : int
        Максимальная глубина дерева
    min_samples_split : int
        Минимальное количество образцов для разбиения
    min_samples_leaf : int
        Минимальное количество образцов в листе
        
    Returns
    -------
    dict
        Словарь с метриками модели
    """
    dt = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    cv_scores = cross_val_score(dt, X_train, y_train, cv=5)
    
    return {
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'depth': dt.get_depth(),
        'n_leaves': dt.get_n_leaves(),
        'feature_importances': dt.feature_importances_
    }

In [10]:
# Интерактивный виджет для экспериментов с Decision Tree
def plot_dt_hyperparameters(max_depth=5, min_samples_split=20, min_samples_leaf=10):
    """
    Интерактивная визуализация влияния гиперпараметров Decision Tree.
    
    Parameters
    ----------
    max_depth : int
        Максимальная глубина дерева
    min_samples_split : int
        Минимальное количество образцов для разбиения
    min_samples_leaf : int
        Минимальное количество образцов в листе
    """
    results = analyze_hyperparameters(max_depth, min_samples_split, min_samples_leaf)
    
    # Создаем subplot с метриками и важностью признаков
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=['Метрики модели', 'Важность признаков'],
        specs=[[{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # График метрик
    metrics = ['Test Accuracy', 'CV Mean']
    values = [results['accuracy'], results['cv_mean']]
    colors = ['blue', 'green']
    
    fig.add_trace(
        go.Bar(x=metrics, y=values,
               text=[f'{v:.3f}' for v in values],
               textposition='auto',
               marker_color=colors,
               name='Метрики'),
        row=1, col=1
    )
    
    # График важности признаков
    fig.add_trace(
        go.Bar(x=[f'Признак_{i+1}' for i in range(len(results['feature_importances']))],
               y=results['feature_importances'],
               text=[f'{imp:.3f}' for imp in results['feature_importances']],
               textposition='auto',
               marker_color='orange',
               name='Важность'),
        row=1, col=2
    )
    
    fig.update_yaxes(title_text="Значение метрики", row=1, col=1)
    fig.update_yaxes(title_text="Важность признака", row=1, col=2)
    
    fig.update_layout(
        title=f'Decision Tree: depth={max_depth}, min_split={min_samples_split}, min_leaf={min_samples_leaf}<br>'
              f'Accuracy: {results["accuracy"]:.3f}, CV: {results["cv_mean"]:.3f}±{results["cv_std"]:.3f}, '
              f'Узлов: {results["depth"]}, Листьев: {results["n_leaves"]}',
        height=400,
        showlegend=False
    )
    
    fig.show()

# Создаем интерактивный виджет
interact(plot_dt_hyperparameters,
         max_depth=IntSlider(min=1, max=20, step=1, value=5),
         min_samples_split=IntSlider(min=2, max=50, step=2, value=20),
         min_samples_leaf=IntSlider(min=1, max=30, step=1, value=10))

interactive(children=(IntSlider(value=5, description='max_depth', max=20, min=1), IntSlider(value=20, descript…

<function __main__.plot_dt_hyperparameters(max_depth=5, min_samples_split=20, min_samples_leaf=10)>

## 5. Теория Bagging

### 5.1 Что такое Bagging?

**Bagging** (Bootstrap Aggregating) — это метод ансамблевого обучения, который:
1. Создает множество bootstrap выборок из исходных данных
2. Обучает базовую модель на каждой выборке  
3. Агрегирует предсказания всех моделей

### 5.2 Bootstrap Sampling

**Bootstrap выборка** — это выборка с возвращением из исходного датасета того же размера.

Если исходный датасет содержит $N$ объектов: $D = \{(x_1, y_1), ..., (x_N, y_N)\}$, то bootstrap выборка $D_b$ формируется следующим образом:
- Выбираем $N$ объектов случайно с возвращением из $D$
- Некоторые объекты могут повториться, другие — не попасть в выборку

**Важное свойство**: Примерно $\frac{1}{e} \approx 36.8\%$ объектов не попадает в bootstrap выборку.

### 5.3 Математическое обоснование

Пусть у нас есть $M$ базовых моделей $h_1, h_2, ..., h_M$, обученных на bootstrap выборках.

Для задачи **классификации** итоговое предсказание:
$$H(x) = \text{majority vote}\{h_1(x), h_2(x), ..., h_M(x)\}$$

Для задачи **регрессии** итоговое предсказание:
$$H(x) = \frac{1}{M} \sum_{i=1}^M h_i(x)$$

### 5.4 Уменьшение дисперсии

Ключевое преимущество Bagging — уменьшение дисперсии предсказаний.

Если базовые модели имеют дисперсию $\sigma^2$ и некоррелированы, то дисперсия ансамбля:
$$\text{Var}(H) = \frac{\sigma^2}{M}$$

На практике модели коррелированы с корреляцией $\rho$, поэтому:
$$\text{Var}(H) = \rho\sigma^2 + \frac{(1-\rho)\sigma^2}{M}$$

### 5.5 Алгоритм Bagging

1. **Вход**: Обучающая выборка $D$, базовый алгоритм $A$, количество моделей $M$
2. **Для** $i = 1, 2, ..., M$:
   - Создать bootstrap выборку $D_i$ из $D$  
   - Обучить модель $h_i = A(D_i)$
3. **Выход**: Ансамблевая модель $H(x) = \text{aggregate}(h_1(x), ..., h_M(x))$

### 5.6 Out-of-Bag (OOB) оценка

Поскольку ~36.8% объектов не попадают в каждую bootstrap выборку, их можно использовать для оценки качества без отдельной валидационной выборки:

$$\text{OOB Error} = \frac{1}{N} \sum_{i=1}^N \mathbb{I}[y_i \neq \text{majority vote of models not trained on } x_i]$$

### 5.7 Плюсы и минусы Bagging

#### Преимущества:
- **Уменьшение overfitting**: За счет усреднения снижается дисперсия
- **Параллелизация**: Модели можно обучать независимо
- **OOB оценка**: Встроенная валидация без дополнительных данных
- **Устойчивость**: Менее чувствителен к выбросам и шуму

#### Недостатки:
- **Не уменьшает bias**: Смещение остается тем же, что у базовой модели
- **Вычислительные затраты**: Нужно обучать $M$ моделей  
- **Потеря интерпретируемости**: Сложнее понять логику ансамбля
- **Эффективен только для high-variance моделей**: Например, глубоких деревьев

## 6. Пошаговый пример Bagging

### 6.1 Демонстрация Bootstrap Sampling

Рассмотрим работу Bootstrap sampling на простом примере.

**Исходные данные:** [1, 2, 3, 4, 5, 6, 7, 8] (N = 8)

**Bootstrap выборка 1:** Случайно выбираем 8 элементов с возвращением
- Индексы: [3, 1, 7, 3, 5, 2, 8, 4]  
- Значения: [4, 2, 8, 4, 6, 3, 8, 5]
- OOB объекты: [1, 7] (25% от исходного размера)

**Bootstrap выборка 2:**
- Индексы: [2, 5, 1, 6, 2, 8, 3, 5]
- Значения: [3, 6, 2, 7, 3, 8, 4, 6]  
- OOB объекты: [1, 5] (25% от исходного размера)

**Bootstrap выборка 3:**
- Индексы: [1, 3, 8, 1, 4, 6, 2, 8]
- Значения: [2, 4, 8, 2, 5, 7, 3, 8]
- OOB объекты: [1, 6] (25% от исходного размера)

### 6.2 Вычисление Out-of-Bag Error

Для каждого объекта исходной выборки используем только те модели, которые **не** видели этот объект в процессе обучения:

- **Объект 1** не попал в выборки 2 и 3 → используем модели M₂, M₃
- **Объект 7** не попал в выборки 1 и 3 → используем модели M₁, M₃  
- **Объект 5** не попал в выборки 1 и 2 → используем модели M₁, M₂
- **Объект 6** не попал в выборки 2 и 3 → используем модели M₂, M₃

$$\text{OOB Error} = \frac{1}{N} \sum_{i=1}^{N} \mathbb{I}[y_i \neq \text{majority vote of OOB models for } x_i]$$

### 6.3 Математический пример уменьшения дисперсии

Предположим, что каждая базовая модель имеет:
- **Точность:** 0.7
- **Дисперсию ошибки:** σ² = 0.21

Для **некоррелированных** моделей дисперсия ансамбля из M моделей:
$$\text{Var}(\text{ensemble}) = \frac{\sigma^2}{M} = \frac{0.21}{M}$$

- M = 1: Var = 0.21
- M = 5: Var = 0.042  
- M = 10: Var = 0.021
- M = 50: Var = 0.0042

Для **коррелированных** моделей с корреляцией ρ = 0.3:
$$\text{Var}(\text{ensemble}) = \rho\sigma^2 + \frac{(1-\rho)\sigma^2}{M} = 0.3 \cdot 0.21 + \frac{0.7 \cdot 0.21}{M}$$

- M = 10: Var = 0.063 + 0.0147 = 0.078
- M = 50: Var = 0.063 + 0.0029 = 0.066

In [11]:
# Создание Bagging классификатора для датасета Breast Cancer
bagging_classifier = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        max_depth=8,            # Достаточная глубина для Breast Cancer
        min_samples_split=5,    # Минимум образцов для разбиения
        min_samples_leaf=2,     # Минимум образцов в листе
        max_features='sqrt'     # Случайный выбор признаков в базовых деревьях
    ),
    n_estimators=50,            # Количество базовых моделей
    max_samples=0.8,           # Используем 80% данных для каждого дерева
    max_features=0.8,          # Используем 80% признаков для каждого дерева
    random_state=42,
    oob_score=True,            # Вычисляем OOB score
    n_jobs=-1                  # Параллельное обучение деревьев
)

print("=== НАСТРОЙКИ BAGGING ===")
print(f"Количество базовых моделей: {bagging_classifier.n_estimators}")
print(f"Процент образцов для каждого дерева: {bagging_classifier.max_samples * 100:.0f}%")
print(f"Процент признаков для каждого дерева: {bagging_classifier.max_features * 100:.0f}%")
print(f"Максимальная глубина базовых деревьев: {bagging_classifier.estimator.max_depth}")

# Обучение модели
bagging_classifier.fit(X_train, y_train)

=== НАСТРОЙКИ BAGGING ===
Количество базовых моделей: 50
Процент образцов для каждого дерева: 80%
Процент признаков для каждого дерева: 80%
Максимальная глубина базовых деревьев: 8


,estimator,DecisionTreeC...mples_split=5)
,n_estimators,50
,max_samples,0.8
,max_features,0.8
,bootstrap,True
,bootstrap_features,False
,oob_score,True
,warm_start,False
,n_jobs,-1
,random_state,42
,verbose,0


In [12]:
# Расширенная оценка Bagging модели
y_pred_bagging = bagging_classifier.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
precision_bagging = precision_score(y_test, y_pred_bagging, average='weighted')
recall_bagging = recall_score(y_test, y_pred_bagging, average='weighted')
f1_bagging = f1_score(y_test, y_pred_bagging, average='weighted')
oob_score = bagging_classifier.oob_score_

print("=== РЕЗУЛЬТАТЫ BAGGING НА BREAST CANCER ===")
print(f"Accuracy:  {accuracy_bagging:.4f}")
print(f"Precision: {precision_bagging:.4f}")
print(f"Recall:    {recall_bagging:.4f}")
print(f"F1-score:  {f1_bagging:.4f}")
print(f"OOB Score: {oob_score:.4f}")

# Сравнение с одиночным деревом для демонстрации улучшений
single_deep_tree = DecisionTreeClassifier(
    max_depth=None,              # Без ограничений для высокой дисперсии
    min_samples_split=2,         # Минимальные ограничения
    min_samples_leaf=1,
    random_state=42
)
single_deep_tree.fit(X_train, y_train)
y_pred_single = single_deep_tree.predict(X_test)
accuracy_single = accuracy_score(y_test, y_pred_single)
f1_single = f1_score(y_test, y_pred_single, average='weighted')

print(f"\n=== СРАВНЕНИЕ С ОДИНОЧНЫМ ГЛУБОКИМ ДЕРЕВОМ ===")
print(f"Single Tree Accuracy: {accuracy_single:.4f}")
print(f"Bagging Accuracy:     {accuracy_bagging:.4f}")
print(f"Улучшение:            {accuracy_bagging - accuracy_single:.4f} ({((accuracy_bagging/accuracy_single - 1) * 100):.1f}%)")
print(f"Снижение overfitting: {abs(oob_score - accuracy_bagging):.4f}")

# Анализ важности признаков в Bagging
bagging_importances = np.mean([estimator.feature_importances_ for estimator in bagging_classifier.estimators_], axis=0)
top_bagging = np.argsort(bagging_importances)[-10:][::-1]

print(f"\n=== ВАЖНОСТЬ ПРИЗНАКОВ В BAGGING (ТОП-10) ===")
for i, feat_idx in enumerate(top_bagging):
    print(f"{i+1}. {feature_names[feat_idx]}: {bagging_importances[feat_idx]:.4f}")

# Расширенная визуализация результатов Bagging
fig_bagging = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Метрики качества', 'Сравнение с одиночным деревом', 
                   'Важность признаков (топ-10)', 'Стабильность предсказаний'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'histogram'}]],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# График 1: Все метрики
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'OOB Score']
values = [accuracy_bagging, precision_bagging, recall_bagging, f1_bagging, oob_score]

fig_bagging.add_trace(
    go.Bar(x=metrics, y=values,
           text=[f'{v:.3f}' for v in values],
           textposition='auto',
           marker_color=[COLOR_PALETTE['primary'], COLOR_PALETTE['success'], 
                        COLOR_PALETTE['warning'], COLOR_PALETTE['danger'], COLOR_PALETTE['info']],
           name='Bagging',
           showlegend=False),
    row=1, col=1
)

# График 2: Сравнение с одиночным деревом
comparison_models = ['Single Deep Tree', 'Bagging Ensemble']
comparison_accuracy = [accuracy_single, accuracy_bagging]
comparison_f1 = [f1_single, f1_bagging]

fig_bagging.add_trace(
    go.Bar(x=comparison_models, y=comparison_accuracy,
           name='Accuracy',
           marker_color=COLOR_PALETTE['primary'],
           text=[f'{v:.3f}' for v in comparison_accuracy],
           textposition='auto'),
    row=1, col=2
)

fig_bagging.add_trace(
    go.Bar(x=comparison_models, y=comparison_f1,
           name='F1-Score',
           marker_color=COLOR_PALETTE['secondary'],
           text=[f'{v:.3f}' for v in comparison_f1],
           textposition='auto'),
    row=1, col=2
)

# График 3: Важность признаков
top_names = [feature_names[i] for i in top_bagging]
top_values = bagging_importances[top_bagging]

fig_bagging.add_trace(
    go.Bar(y=top_names, x=top_values, orientation='h',
           text=[f'{v:.3f}' for v in top_values],
           textposition='auto',
           marker_color=COLOR_PALETTE['success'],
           name='Важность',
           showlegend=False),
    row=2, col=1
)

# График 4: Анализ стабильности (дисперсии предсказаний между деревьями)
# Безопасное получение предсказаний от всех деревьев
try:
    all_predictions = np.array([estimator.predict(X_test) for estimator in bagging_classifier.estimators_])
    prediction_variance = np.var(all_predictions, axis=0)
    
    fig_bagging.add_trace(
        go.Histogram(x=prediction_variance,
                    nbinsx=30,
                    name='Дисперсия предсказаний',
                    marker_color=COLOR_PALETTE['warning'],
                    showlegend=False),
        row=2, col=2
    )
except Exception as e:
    # Если возникает ошибка размерности, показываем распределение вероятностей обычных предсказаний
    print(f"Ошибка при анализе стабильности: {e}")
    fig_bagging.add_trace(
        go.Histogram(x=y_pred_bagging,
                    nbinsx=10,
                    name='Распределение предсказаний',
                    marker_color=COLOR_PALETTE['warning'],
                    showlegend=False),
        row=2, col=2
    )

fig_bagging.update_layout(
    title=f'Детальный анализ Bagging на Breast Cancer датасете<br>'
          f'Улучшение accuracy: +{((accuracy_bagging/accuracy_single - 1) * 100):.1f}% относительно одиночного дерева',
    height=800,
    showlegend=True
)

# Обновление осей
fig_bagging.update_yaxes(title_text="Значение метрики", row=1, col=1)
fig_bagging.update_yaxes(title_text="Значение метрики", row=1, col=2)
fig_bagging.update_xaxes(title_text="Важность признака", row=2, col=1)
fig_bagging.update_xaxes(title_text="Дисперсия предсказаний", row=2, col=2)
fig_bagging.update_yaxes(title_text="Частота", row=2, col=2)

fig_bagging.show()

=== РЕЗУЛЬТАТЫ BAGGING НА BREAST CANCER ===
Accuracy:  0.9298
Precision: 0.9298
Recall:    0.9298
F1-score:  0.9298
OOB Score: 0.9648

=== СРАВНЕНИЕ С ОДИНОЧНЫМ ГЛУБОКИМ ДЕРЕВОМ ===
Single Tree Accuracy: 0.9181
Bagging Accuracy:     0.9298
Улучшение:            0.0117 (1.3%)
Снижение overfitting: 0.0350

=== ВАЖНОСТЬ ПРИЗНАКОВ В BAGGING (ТОП-10) ===
1. concavity error: 0.0784
2. compactness error: 0.0623
3. smoothness error: 0.0596
4. worst perimeter: 0.0578
5. symmetry error: 0.0575
6. mean area: 0.0538
7. worst texture: 0.0508
8. radius error: 0.0504
9. mean concavity: 0.0481
10. texture error: 0.0480
Ошибка при анализе стабильности: X has 30 features, but DecisionTreeClassifier is expecting 24 features as input.


In [13]:
# Сравнение с одиночным глубоким деревом
single_deep_tree = DecisionTreeClassifier(max_depth=None, random_state=42)
single_deep_tree.fit(X_train, y_train)
y_pred_single = single_deep_tree.predict(X_test)
accuracy_single = accuracy_score(y_test, y_pred_single)

print(f"\n=== СРАВНЕНИЕ ===")
print(f"Одиночное глубокое дерево: {accuracy_single:.4f}")
print(f"Bagging (50 деревьев):     {accuracy_bagging:.4f}")
print(f"Улучшение:                 {accuracy_bagging - accuracy_single:.4f}")


=== СРАВНЕНИЕ ===
Одиночное глубокое дерево: 0.9181
Bagging (50 деревьев):     0.9298
Улучшение:                 0.0117


## 7. Теория Random Forest

### 7.1 Что такое Random Forest?

**Random Forest** — это усовершенствованная версия Bagging, которая вводит дополнительную случайность при выборе признаков для разбиения в каждом узле дерева.

### 7.2 Ключевые отличия от Bagging

1. **Bootstrap sampling** — как в Bagging
2. **Случайный выбор признаков** — в каждом узле рассматривается только случайное подмножество признаков

### 7.3 Алгоритм Random Forest

1. **Для каждого дерева** $t = 1, 2, ..., T$:
   - Создать bootstrap выборку $D_t$ размера $N$
   - **Для каждого узла дерева**:
     - Случайно выбрать $m \ll d$ признаков из всех $d$ признаков  
     - Найти лучшее разбиение среди этих $m$ признаков
     - Разделить узел по найденному признаку
2. **Агрегировать предсказания** всех деревьев

### 7.4 Выбор количества признаков

Рекомендуемые значения для $m$:
- **Классификация**: $m = \sqrt{d}$  
- **Регрессия**: $m = \frac{d}{3}$

где $d$ — общее количество признаков.

### 7.5 Математическое обоснование

Random Forest уменьшает корреляцию между деревьями, что приводит к лучшему снижению дисперсии.

Если корреляция между деревьями в Bagging равна $\rho$, то в Random Forest она становится меньше: $\rho_{RF} < \rho_{Bagging}$

Следовательно:
$$\text{Var}(RF) = \rho_{RF}\sigma^2 + \frac{(1-\rho_{RF})\sigma^2}{T} < \text{Var}(Bagging)$$

### 7.6 Out-of-Bag Error в Random Forest

OOB error вычисляется аналогично Bagging, но учитывает случайный выбор признаков:

Для объекта $x_i$ используются только те деревья, которые:
1. Не содержали $x_i$ в обучающей выборке  
2. В момент предсказания имеют доступ к нужным признакам

### 7.7 Важность признаков

Random Forest предоставляет два способа измерения важности признаков:

#### 7.7.1 Mean Decrease Impurity (MDI)
$$Importance_j = \frac{1}{T} \sum_{t=1}^T \sum_{nodes} w_t \cdot \Delta I_t \cdot \mathbb{I}[feature_t = j]$$

где:
- $w_t$ — доля объектов в узле $t$  
- $\Delta I_t$ — уменьшение impurity в узле $t$
- $\mathbb{I}[feature_t = j]$ — индикатор использования признака $j$

#### 7.7.2 Mean Decrease Accuracy (MDA)  
1. Вычислить OOB accuracy исходной модели
2. Для каждого признака $j$:
   - Перемешать значения признака $j$ в OOB выборках
   - Вычислить новый OOB accuracy
   - $Importance_j = OOB_{original} - OOB_{permuted}$

### 7.8 Плюсы и минусы Random Forest

#### Преимущества:
- **Меньше переобучения** по сравнению с одиночными деревьями
- **Лучше чем Bagging** за счет уменьшения корреляции
- **Встроенная валидация** через OOB error
- **Важность признаков** автоматически вычисляется
- **Работает "из коробки"** — мало гиперпараметров для настройки
- **Робустность** к выбросам и шуму

#### Недостатки:
- **Потеря интерпретируемости** по сравнению с одиночным деревом
- **Может переобучаться** на очень зашумленных данных
- **Смещение к многозначным признакам** в измерении важности  
- **Большие требования к памяти** для хранения всех деревьев
- **Не очень хорош** для экстраполяции за пределы обучающих данных

## 8. Практическая реализация Random Forest

In [14]:
# Создание Random Forest классификатора
rf_classifier = RandomForestClassifier(
    n_estimators=150,              # Оптимальное количество деревьев для Breast Cancer
    max_features='sqrt',           # sqrt(n_features) - оптимально для медицинских данных  
    max_depth=8,                   # Ограниченная глубина для баланса bias-variance
    min_samples_split=5,           # Минимум образцов для разбиения
    min_samples_leaf=2,            # Минимум образцов в листе
    max_samples=0.8,               # Используем 80% данных для каждого дерева
    bootstrap=True,                # Использовать bootstrap выборки
    oob_score=True,                # Вычислять OOB score
    class_weight='balanced',       # Балансировка классов
    random_state=42,
    n_jobs=-1                      # Использовать все процессоры
)

print("=== НАСТРОЙКИ RANDOM FOREST ===")
print(f"Количество деревьев: {rf_classifier.n_estimators}")
print(f"Максимальная глубина: {rf_classifier.max_depth}")
print(f"Количество признаков для разбиения: {rf_classifier.max_features}")
print(f"Минимум образцов для разбиения: {rf_classifier.min_samples_split}")
print(f"Минимум образцов в листе: {rf_classifier.min_samples_leaf}")
print(f"Процент образцов для каждого дерева: {rf_classifier.max_samples * 100:.0f}%")
print(f"Балансировка классов: {rf_classifier.class_weight}")

# Обучение модели
rf_classifier.fit(X_train, y_train)

=== НАСТРОЙКИ RANDOM FOREST ===
Количество деревьев: 150
Максимальная глубина: 8
Количество признаков для разбиения: sqrt
Минимум образцов для разбиения: 5
Минимум образцов в листе: 2
Процент образцов для каждого дерева: 80%
Балансировка классов: balanced


,n_estimators,150
,criterion,'gini'
,max_depth,8
,min_samples_split,5
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,True


In [15]:
# Расширенная оценка Random Forest
y_pred_rf = rf_classifier.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
oob_score_rf = rf_classifier.oob_score_

print("=== РЕЗУЛЬТАТЫ RANDOM FOREST НА BREAST CANCER ===")
print(f"Accuracy:  {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall:    {recall_rf:.4f}")
print(f"F1-score:  {f1_rf:.4f}")
print(f"OOB Score: {oob_score_rf:.4f}")

# Сравнение с Bagging для демонстрации преимуществ feature selection
bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=8, min_samples_split=5, min_samples_leaf=2),
    n_estimators=150,
    max_samples=0.8,
    max_features=1.0,  # Используем все признаки (без random feature selection)
    random_state=42,
    oob_score=True,
    n_jobs=-1
)
bagging_model.fit(X_train, y_train)
y_pred_bagging_comp = bagging_model.predict(X_test)
accuracy_bagging_comp = accuracy_score(y_test, y_pred_bagging_comp)
f1_bagging_comp = f1_score(y_test, y_pred_bagging_comp, average='weighted')

print(f"\n=== СРАВНЕНИЕ С BAGGING (БЕЗ FEATURE SELECTION) ===")
print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print(f"Bagging Accuracy:       {accuracy_bagging_comp:.4f}")
print(f"Преимущество RF:        {accuracy_rf - accuracy_bagging_comp:.4f} ({((accuracy_rf/accuracy_bagging_comp - 1) * 100):.1f}%)")

# Анализ важности признаков в Random Forest
rf_importances = rf_classifier.feature_importances_
top_rf = np.argsort(rf_importances)[-10:][::-1]

print(f"\n=== ВАЖНОСТЬ ПРИЗНАКОВ В RANDOM FOREST (ТОП-10) ===")
for i, feat_idx in enumerate(top_rf):
    print(f"{i+1}. {feature_names[feat_idx]}: {rf_importances[feat_idx]:.4f}")

# Сравнение важности признаков между RF и Bagging
bagging_importances = np.mean([estimator.feature_importances_ for estimator in bagging_model.estimators_], axis=0)
correlation = np.corrcoef(rf_importances, bagging_importances)[0, 1]
print(f"\nКорреляция важности признаков между RF и Bagging: {correlation:.3f}")

# Расширенная визуализация Random Forest
fig_rf = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Метрики качества', 'RF vs Bagging vs Single Tree', 
                   'Сравнение важности признаков', 'Анализ Feature Selection'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'bar'}]],
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# График 1: Все метрики RF
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'OOB Score']
values = [accuracy_rf, precision_rf, recall_rf, f1_rf, oob_score_rf]

fig_rf.add_trace(
    go.Bar(x=metrics, y=values,
           text=[f'{v:.3f}' for v in values],
           textposition='auto',
           marker_color=[COLOR_PALETTE['primary'], COLOR_PALETTE['success'], 
                        COLOR_PALETTE['warning'], COLOR_PALETTE['danger'], COLOR_PALETTE['info']],
           name='Random Forest',
           showlegend=False),
    row=1, col=1
)

# График 2: Сравнение алгоритмов
algorithms = ['Random Forest', 'Bagging', 'Single Tree']
accuracies = [accuracy_rf, accuracy_bagging_comp, accuracy_single]
f1_scores = [f1_rf, f1_bagging_comp, f1_single]

fig_rf.add_trace(
    go.Bar(x=algorithms, y=accuracies,
           name='Accuracy',
           marker_color=COLOR_PALETTE['primary'],
           text=[f'{v:.3f}' for v in accuracies],
           textposition='auto'),
    row=1, col=2
)

fig_rf.add_trace(
    go.Bar(x=algorithms, y=f1_scores,
           name='F1-Score',
           marker_color=COLOR_PALETTE['secondary'],
           text=[f'{v:.3f}' for v in f1_scores],
           textposition='auto'),
    row=1, col=2
)

# График 3: Сравнение важности признаков RF vs Bagging
fig_rf.add_trace(
    go.Scatter(x=rf_importances, y=bagging_importances,
              mode='markers+text',
              text=[feature_names[i][:8] + '...' if len(feature_names[i]) > 8 else feature_names[i] 
                    for i in range(len(feature_names))],
              textposition='top center',
              marker=dict(size=8, color=COLOR_PALETTE['danger'], opacity=0.6),
              name=f'Корреляция: {correlation:.3f}',
              showlegend=False),
    row=2, col=1
)

# Добавляем линию идеальной корреляции
max_imp = max(max(rf_importances), max(bagging_importances))
fig_rf.add_trace(
    go.Scatter(x=[0, max_imp], y=[0, max_imp],
              mode='lines',
              line=dict(dash='dash', color='gray'),
              name='Идеальная корреляция',
              showlegend=False),
    row=2, col=1
)

# График 4: Эффект количества используемых признаков
feature_counts = ['sqrt', 'log2', '0.3', '0.5', '1.0']
feature_values = [np.sqrt(len(feature_names)), np.log2(len(feature_names)), 
                 0.3*len(feature_names), 0.5*len(feature_names), len(feature_names)]

# Быстрое тестирование разных значений max_features
feature_accuracies = []
for feat_val in ['sqrt', 'log2', 0.3, 0.5, 1.0]:
    rf_temp = RandomForestClassifier(
        n_estimators=50, max_features=feat_val, max_depth=12,
        random_state=42, n_jobs=-1
    )
    rf_temp.fit(X_train, y_train)
    acc_temp = accuracy_score(y_test, rf_temp.predict(X_test))
    feature_accuracies.append(acc_temp)

fig_rf.add_trace(
    go.Bar(x=feature_counts, y=feature_accuracies,
           text=[f'{v:.3f}' for v in feature_accuracies],
           textposition='auto',
           marker_color=COLOR_PALETTE['warning'],
           name='Accuracy по max_features',
           showlegend=False),
    row=2, col=2
)

fig_rf.update_layout(
    title=f'Детальный анализ Random Forest<br>'
          f'Преимущество перед Bagging: +{((accuracy_rf/accuracy_bagging_comp - 1) * 100):.1f}% '
          f'благодаря feature selection',
    height=800,
    showlegend=True
)

# Обновление осей
fig_rf.update_yaxes(title_text="Значение метрики", row=1, col=1)
fig_rf.update_yaxes(title_text="Значение метрики", row=1, col=2)
fig_rf.update_xaxes(title_text="Важность в RF", row=2, col=1)
fig_rf.update_yaxes(title_text="Важность в Bagging", row=2, col=1)
fig_rf.update_xaxes(title_text="max_features", row=2, col=2)
fig_rf.update_yaxes(title_text="Accuracy", row=2, col=2)

fig_rf.show()

=== РЕЗУЛЬТАТЫ RANDOM FOREST НА BREAST CANCER ===
Accuracy:  0.9298
Precision: 0.9298
Recall:    0.9298
F1-score:  0.9298
OOB Score: 0.9623

=== СРАВНЕНИЕ С BAGGING (БЕЗ FEATURE SELECTION) ===
Random Forest Accuracy: 0.9298
Bagging Accuracy:       0.9474
Преимущество RF:        -0.0175 (-1.9%)

=== ВАЖНОСТЬ ПРИЗНАКОВ В RANDOM FOREST (ТОП-10) ===
1. worst area: 0.1515
2. worst perimeter: 0.1373
3. worst concave points: 0.1120
4. mean concave points: 0.0999
5. worst radius: 0.0781
6. mean radius: 0.0642
7. mean perimeter: 0.0614
8. mean concavity: 0.0593
9. worst concavity: 0.0350
10. mean area: 0.0240

Корреляция важности признаков между RF и Bagging: 0.782

=== СРАВНЕНИЕ С BAGGING (БЕЗ FEATURE SELECTION) ===
Random Forest Accuracy: 0.9298
Bagging Accuracy:       0.9474
Преимущество RF:        -0.0175 (-1.9%)

=== ВАЖНОСТЬ ПРИЗНАКОВ В RANDOM FOREST (ТОП-10) ===
1. worst area: 0.1515
2. worst perimeter: 0.1373
3. worst concave points: 0.1120
4. mean concave points: 0.0999
5. worst radius

In [16]:
# Анализ важности признаков
feature_importance = rf_classifier.feature_importances_
print(f"\n=== ВАЖНОСТЬ ПРИЗНАКОВ ===")
for i, importance in enumerate(feature_importance):
    print(f"Признак_{i+1}: {importance:.4f}")

# Визуализация важности признаков Random Forest
fig_importance_rf = go.Figure()

fig_importance_rf.add_trace(
    go.Bar(x=[f'Признак_{i+1}' for i in range(len(feature_importance))],
           y=feature_importance,
           text=[f'{imp:.3f}' for imp in feature_importance],
           textposition='auto',
           marker_color=COLOR_PALETTE['success'],
           name='Важность признаков')
)

fig_importance_rf.update_layout(
    title='Важность признаков в Random Forest',
    xaxis_title='Признаки',
    yaxis_title='Важность',
    height=400
)

fig_importance_rf.show()


=== ВАЖНОСТЬ ПРИЗНАКОВ ===
Признак_1: 0.0642
Признак_2: 0.0104
Признак_3: 0.0614
Признак_4: 0.0240
Признак_5: 0.0093
Признак_6: 0.0196
Признак_7: 0.0593
Признак_8: 0.0999
Признак_9: 0.0043
Признак_10: 0.0044
Признак_11: 0.0118
Признак_12: 0.0035
Признак_13: 0.0077
Признак_14: 0.0238
Признак_15: 0.0044
Признак_16: 0.0011
Признак_17: 0.0041
Признак_18: 0.0035
Признак_19: 0.0025
Признак_20: 0.0025
Признак_21: 0.0781
Признак_22: 0.0143
Признак_23: 0.1373
Признак_24: 0.1515
Признак_25: 0.0131
Признак_26: 0.0198
Признак_27: 0.0350
Признак_28: 0.1120
Признак_29: 0.0115
Признак_30: 0.0059


In [17]:
# Функция для анализа параметров Random Forest на сложном датасете
def analyze_rf_parameters(n_estimators, max_features, max_depth):
    """
    Анализирует влияние гиперпараметров Random Forest
    """
    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth if max_depth > 0 else None,
        min_samples_split=8,        
        min_samples_leaf=3,
        class_weight='balanced',    # Для работы с несбалансированными классами
        random_state=42,
        oob_score=True,
        n_jobs=-1
    )
    
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    oob_score = rf.oob_score_
    cv_scores = cross_val_score(rf, X_train, y_train, cv=5)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'oob_score': oob_score,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'feature_importances': rf.feature_importances_,
        'n_classes': len(np.unique(y_test))
    }

In [18]:
# Упрощенный интерактивный Random Forest анализ
def analyze_rf_simplified(n_estimators=100, max_features='sqrt', max_depth=10):
    """Упрощенная версия анализа Random Forest."""
    
    # Создание и оценка модели
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth if max_depth > 0 else None,
        random_state=42,
        oob_score=True
    )
    
    result = evaluate_model(rf_model, X_train, X_test, y_train, y_test, "Random Forest")
    
    # Создание визуализации
    fig = create_metrics_chart(pd.DataFrame([result]))
    fig.update_layout(
        title=f'Random Forest: n_est={n_estimators}, max_feat={max_features}, depth={max_depth}<br>'
              f'Accuracy: {result["Test Accuracy"]:.3f} | OOB: {result["OOB Score"]:.3f}',
        height=400
    )
    fig.show()
    
    # Важность признаков
    fig_imp = create_importance_chart(rf_model, feature_names, 
                                    f'Важность признаков (топ-10)')
    if fig_imp:
        fig_imp.show()
        
    # Краткий отчет
    print(f"Результаты: Accuracy={result['Test Accuracy']:.4f}, "
          f"OOB={result['OOB Score']:.4f}, CV={result['CV Mean']:.4f}±{result['CV Std']:.4f}")

# Упрощенный виджет
interact(analyze_rf_simplified,
         n_estimators=IntSlider(min=10, max=200, step=20, value=100, description='Деревья'),
         max_features=Dropdown(options=['sqrt', 'log2', 0.5, 1.0], value='sqrt', description='Признаки'),
         max_depth=IntSlider(min=0, max=20, step=2, value=10, description='Глубина'))

interactive(children=(IntSlider(value=100, description='Деревья', max=200, min=10, step=20), Dropdown(descript…

<function __main__.analyze_rf_simplified(n_estimators=100, max_features='sqrt', max_depth=10)>

### 8.1 Интерактивное сравнение влияния гиперпараметров на модели

Следующий интерактивный элемент демонстрирует, как различные гиперпараметры влияют на важность признаков и метрики качества для разных алгоритмов:

In [19]:
# Упрощенное интерактивное сравнение гиперпараметров
def compare_algorithms(algorithm='Random Forest', n_estimators=100, max_depth=10):
    """Упрощенное сравнение алгоритмов с основными параметрами."""
    
    # Создаем выбранную модель
    if algorithm == 'Decision Tree':
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
    elif algorithm == 'Random Forest':
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                     random_state=42, oob_score=True)
    else:  # Bagging
        model = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=max_depth),
                                 n_estimators=n_estimators, random_state=42, oob_score=True)
    
    # Оценка модели
    result = evaluate_model(model, X_train, X_test, y_train, y_test, algorithm)
    
    # Создаем быструю визуализацию
    fig = go.Figure()
    
    metrics = ['Test Accuracy', 'Precision', 'Recall', 'F1-Score', 'CV Mean']
    values = [result[metric] for metric in metrics]
    
    fig.add_trace(go.Bar(
        x=metrics, y=values,
        text=[f'{v:.3f}' for v in values],
        textposition='outside',
        marker_color=COLOR_PALETTE['primary']
    ))
    
    oob_text = f" | OOB: {result['OOB Score']:.3f}" if result['OOB Score'] else ""
    fig.update_layout(
        title=f'{algorithm} - Test Accuracy: {result["Test Accuracy"]:.3f}{oob_text}',
        yaxis_title='Значение метрики',
        height=400
    )
    
    fig.show()
    
    # Анализ важности признаков (если доступен)
    if hasattr(model, 'feature_importances_') or hasattr(model, 'estimators_'):
        fig_imp = create_importance_chart(model, feature_names, f'{algorithm} - Важность признаков')
        if fig_imp:
            fig_imp.show()

# Создаем упрощенный интерактивный виджет
interact(compare_algorithms,
         algorithm=Dropdown(options=['Decision Tree', 'Random Forest', 'Bagging'], 
                           value='Random Forest', description='Алгоритм'),
         n_estimators=IntSlider(min=10, max=200, step=10, value=100, description='Деревья'),
         max_depth=IntSlider(min=0, max=20, step=2, value=10, description='Глубина'))

interactive(children=(Dropdown(description='Алгоритм', index=1, options=('Decision Tree', 'Random Forest', 'Ba…

<function __main__.compare_algorithms(algorithm='Random Forest', n_estimators=100, max_depth=10)>

## 9. Сравнительный анализ: Decision Tree vs Bagging vs Random Forest

In [20]:
# Определение оптимизированных моделей
models = {
    'Decision Tree': DecisionTreeClassifier(
        max_depth=8,
        min_samples_split=15,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42
    ),
    'Decision Tree (глубокое)': DecisionTreeClassifier(
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        random_state=42
    ),
    'Bagging': BaggingClassifier(
        estimator=DecisionTreeClassifier(
            max_depth=10,
            min_samples_split=10,
            min_samples_leaf=3,
            max_features='sqrt'
        ),
        n_estimators=100,
        max_samples=0.8,
        max_features=0.8,
        random_state=42,
        oob_score=True,
        n_jobs=-1
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_features='log2',
        max_depth=12,
        min_samples_split=8,
        min_samples_leaf=3,
        max_samples=0.8,
        class_weight='balanced',
        bootstrap=True,
        oob_score=True,
        random_state=42,
        n_jobs=-1
    )
}

print("=== КОНФИГУРАЦИЯ МОДЕЛЕЙ ===")
for name, model in models.items():
    print(f"\n{name}:")
    key_params = {}
    if hasattr(model, 'max_depth'):
        key_params['max_depth'] = model.max_depth
    if hasattr(model, 'n_estimators'):
        key_params['n_estimators'] = model.n_estimators
    if hasattr(model, 'max_features'):
        key_params['max_features'] = model.max_features
    if hasattr(model, 'class_weight'):
        key_params['class_weight'] = model.class_weight
    if hasattr(model, 'estimator'):  # для ensemble методов
        if hasattr(model.estimator, 'max_depth'):
            key_params['base_max_depth'] = model.estimator.max_depth
    
    for param, value in key_params.items():
        print(f"  {param}: {value}")

=== КОНФИГУРАЦИЯ МОДЕЛЕЙ ===

Decision Tree:
  max_depth: 8
  max_features: sqrt
  class_weight: None

Decision Tree (глубокое):
  max_depth: None
  max_features: None
  class_weight: None

Bagging:
  n_estimators: 100
  max_features: 0.8
  base_max_depth: 10

Random Forest:
  max_depth: 12
  n_estimators: 200
  max_features: log2
  class_weight: balanced
  base_max_depth: None


In [21]:
# Обучение и оценка всех моделей (использует helper функции)
results = []

for name, model in models.items():
    print(f"Обучение {name}...")
    result = evaluate_model(model, X_train, X_test, y_train, y_test, name)
    results.append(result)

results_df = pd.DataFrame(results)
print("\n=== СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ ===")
print(results_df.round(4).to_string(index=False))

# Интерактивная сводная таблица
fig_summary = create_comparison_table(results_df)
fig_summary.show()

Обучение Decision Tree...
Обучение Decision Tree (глубокое)...
Обучение Bagging...
Обучение Random Forest...
Обучение Random Forest...

=== СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ ===
                  Модель  Test Accuracy  Precision  Recall  F1-Score  CV Mean  CV Std  OOB Score
           Decision Tree         0.9357     0.9359  0.9357    0.9358   0.9197  0.0230        NaN
Decision Tree (глубокое)         0.9181     0.9181  0.9181    0.9181   0.9247  0.0369        NaN
                 Bagging         0.9415     0.9414  0.9415    0.9413   0.9624  0.0285     0.9573
           Random Forest         0.9298     0.9305  0.9298    0.9300   0.9700  0.0232     0.9573

=== СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ ===
                  Модель  Test Accuracy  Precision  Recall  F1-Score  CV Mean  CV Std  OOB Score
           Decision Tree         0.9357     0.9359  0.9357    0.9358   0.9197  0.0230        NaN
Decision Tree (глубокое)         0.9181     0.9181  0.9181    0.9181   0.9247  0.0369        NaN
               

In [22]:
# Комплексный анализ результатов
print_model_summary(results_df)

# Создаем основные визуализации
fig_metrics = create_metrics_chart(results_df)
fig_metrics.show()

# Анализ важности признаков для лучшей модели
best_model_name = results_df.loc[results_df['Test Accuracy'].idxmax()]['Модель']
best_model_obj = models[best_model_name]

fig_importance = create_importance_chart(best_model_obj, feature_names, 
                                       f"Важность признаков - {best_model_name}")
if fig_importance:
    fig_importance.show()

# Дополнительный анализ производительности vs стабильности
fig_scatter = go.Figure()

fig_scatter.add_trace(
    go.Scatter(x=results_df['CV Mean'], 
               y=results_df['CV Std'],
               mode='markers+text',
               text=results_df['Модель'],
               textposition='top center',
               marker=dict(
                   size=15,
                   color=results_df['Test Accuracy'],
                   colorscale='Viridis',
                   showscale=True,
                   colorbar=dict(title="Test Accuracy")
               ),
               name='Модели')
)

fig_scatter.update_layout(
    title='Производительность vs Стабильность моделей',
    xaxis_title='CV Mean (Производительность)',
    yaxis_title='CV Std (Нестабильность)',
    height=500
)

fig_scatter.show()

КРАТКИЙ АНАЛИЗ РЕЗУЛЬТАТОВ
Лучшая модель: Bagging (Accuracy: 0.9415)
Самая стабильная: Decision Tree (CV Std: 0.0230)

ПРЕИМУЩЕСТВА АНСАМБЛЕВЫХ МЕТОДОВ:
   Bagging: +0.6% улучшение accuracy
   Random Forest: +-0.6% улучшение accuracy


In [23]:
# Упрощенный анализ важности признаков между методами
rf_names, rf_values = analyze_feature_importance(models['Random Forest'], feature_names)
bagging_names, bagging_values = analyze_feature_importance(models['Bagging'], feature_names)

if rf_names and bagging_names and len(rf_values) == len(bagging_values):
    # Корреляция между методами
    correlation = np.corrcoef(rf_values, bagging_values)[0, 1]
    print(f"Корреляция важности признаков между RF и Bagging: {correlation:.4f}")
    
    # Сравнительная визуализация
    fig = go.Figure()
    
    # Топ-10 признаков по средней важности
    avg_importance = (rf_values + bagging_values) / 2
    top_indices = np.argsort(avg_importance)[-10:][::-1]
    
    fig.add_trace(go.Bar(
        name='Random Forest',
        x=[feature_names[i] for i in top_indices],
        y=rf_values[top_indices],
        marker_color=COLOR_PALETTE['primary']
    ))
    
    fig.add_trace(go.Bar(
        name='Bagging',
        x=[feature_names[i] for i in top_indices],
        y=bagging_values[top_indices],
        marker_color=COLOR_PALETTE['secondary']
    ))
    
    fig.update_layout(
        title=f'Сравнение важности признаков (корреляция: {correlation:.3f})',
        barmode='group',
        height=500
    )
    
    fig.show()
    
    # Топ-5 признаков
    print("\nТоп-5 признаков по средней важности:")
    for i, idx in enumerate(top_indices[:5]):
        avg_imp = avg_importance[idx]
        print(f"{i+1}. {feature_names[idx]}: RF={rf_values[idx]:.4f}, Bagging={bagging_values[idx]:.4f}, Ср={avg_imp:.4f}")

else:
    print("Используем только Random Forest важности:")
    fig_rf = create_importance_chart(models['Random Forest'], feature_names)
    if fig_rf:
        fig_rf.show()

Корреляция важности признаков между RF и Bagging: 0.9632



Топ-5 признаков по средней важности:
1. mean radius: RF=0.1365, Bagging=0.0800, Ср=0.1082
2. mean texture: RF=0.1355, Bagging=0.0720, Ср=0.1037
3. mean perimeter: RF=0.1031, Bagging=0.0586, Ср=0.0808
4. mean area: RF=0.0915, Bagging=0.0567, Ср=0.0741
5. mean smoothness: RF=0.0689, Bagging=0.0566, Ср=0.0628


In [24]:
# Создание совместимой Bagging модели для детального анализа
print("Создаем совместимую Bagging модель для детального анализа...")

compatible_bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=8, min_samples_split=5, min_samples_leaf=2),
    n_estimators=100,
    max_samples=0.8,
    random_state=42,
    n_jobs=-1
)

compatible_bagging.fit(X_train, y_train)

# Получаем важности для детального сравнения
rf_importances = models['Random Forest'].feature_importances_
bagging_importances = np.mean([est.feature_importances_ for est in compatible_bagging.estimators_], axis=0)

print(f"RF размерность: {len(rf_importances)}")
print(f"Bagging размерность: {len(bagging_importances)}")

if len(rf_importances) == len(bagging_importances):
    print(f"Совместимые размерности: {len(bagging_importances)} признаков")
    
    # Детальное сравнение методов
    correlation = np.corrcoef(rf_importances, bagging_importances)[0, 1]
    
    # Создаем комплексную визуализацию
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Сравнение методов', 'Корреляция важности', 
                       'Распределение важности', 'Топ-5 признаков'],
        specs=[[{'type': 'bar'}, {'type': 'scatter'}],
               [{'type': 'histogram'}, {'type': 'bar'}]]
    )
    
    # График 1: Сравнение топ-10 признаков
    avg_importance = (rf_importances + bagging_importances) / 2
    top_indices = np.argsort(avg_importance)[-10:][::-1]
    
    fig.add_trace(go.Bar(name='RF', x=top_indices, y=rf_importances[top_indices],
                        marker_color=COLOR_PALETTE['primary']), row=1, col=1)
    fig.add_trace(go.Bar(name='Bagging', x=top_indices, y=bagging_importances[top_indices],
                        marker_color=COLOR_PALETTE['secondary']), row=1, col=1)
    
    # График 2: Scatter plot корреляции
    fig.add_trace(go.Scatter(x=bagging_importances, y=rf_importances, mode='markers',
                            name=f'Корреляция: {correlation:.3f}',
                            marker_color=COLOR_PALETTE['accent']), row=1, col=2)
    
    # График 3: Распределение важности RF
    fig.add_trace(go.Histogram(x=rf_importances, name='RF распределение',
                              marker_color=COLOR_PALETTE['info']), row=2, col=1)
    
    # График 4: Топ-5 признаков
    top_5_indices = top_indices[:5]
    fig.add_trace(go.Bar(x=top_5_indices, y=avg_importance[top_5_indices],
                        text=[f'П_{i+1}' for i in top_5_indices],
                        name='Средняя важность',
                        marker_color=COLOR_PALETTE['success']), row=2, col=2)
    
    fig.update_layout(
        title=f'Анализ важности признаков<br>Корреляция между методами: {correlation:.4f}',
        height=800,
        showlegend=True
    )
    
    fig.show()
    
    print(f"\nКорреляция важности признаков: {correlation:.4f}")
    print(f"\nТоп-5 признаков по средней важности:")
    for i, idx in enumerate(top_5_indices):
        print(f"{i+1}. Признак_{idx+1}: Bagging={bagging_importances[idx]:.4f}, RF={rf_importances[idx]:.4f}")

else:
    print("Ошибка размерностей - создание совместимых моделей")

Создаем совместимую Bagging модель для детального анализа...
RF размерность: 30
Bagging размерность: 30
Совместимые размерности: 30 признаков



Корреляция важности признаков: 0.7509

Топ-5 признаков по средней важности:
1. Признак_23: Bagging=0.2778, RF=0.1355
2. Признак_28: Bagging=0.3003, RF=0.1031
3. Признак_24: Bagging=0.1004, RF=0.1365
4. Признак_21: Bagging=0.1485, RF=0.0658
5. Признак_8: Bagging=0.0743, RF=0.0915


In [25]:
# Статистический анализ важности признаков (финальный)
print("СТАТИСТИЧЕСКИЙ АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ")
print("=" * 50)

# Используем совместимую модель для анализа
if 'compatible_bagging' in locals():
    bagging_importances = np.mean([est.feature_importances_ for est in compatible_bagging.estimators_], axis=0)
    rf_importances = models['Random Forest'].feature_importances_
    
    # Статистики
    print(f"\nBagging Classifier:")
    print(f"  Среднее: {np.mean(bagging_importances):.4f}")
    print(f"  Медиана: {np.median(bagging_importances):.4f}")
    print(f"  Стд. отклонение: {np.std(bagging_importances):.4f}")
    
    print(f"\nRandom Forest Classifier:")
    print(f"  Среднее: {np.mean(rf_importances):.4f}")
    print(f"  Медиана: {np.median(rf_importances):.4f}")
    print(f"  Стд. отклонение: {np.std(rf_importances):.4f}")
    
    # Корреляция
    correlation = np.corrcoef(bagging_importances, rf_importances)[0, 1]
    print(f"\nКорреляция важности между методами: {correlation:.4f}")
    
    # Топ-10 признаков
    avg_importance = (bagging_importances + rf_importances) / 2
    top_indices = np.argsort(avg_importance)[-10:][::-1]
    
    print(f"\nТоп-10 признаков по средней важности:")
    print("-" * 70)
    print(f"{'Ранг':<4} {'Признак':<12} {'Bagging':<10} {'RF':<10} {'Среднее':<10}")
    print("-" * 70)
    
    for i, idx in enumerate(top_indices):
        feature_name = f"Признак_{idx+1}"
        print(f"{i+1:<4} {feature_name:<12} {bagging_importances[idx]:<10.4f} {rf_importances[idx]:<10.4f} {avg_importance[idx]:<10.4f}")
    
    # Упрощенная интерактивная таблица
    comparison_data = []
    for i, (bag_imp, rf_imp) in enumerate(zip(bagging_importances, rf_importances)):
        comparison_data.append([
            i+1,  # Ранг
            f'Признак_{i+1}',  # Название
            f'{bag_imp:.4f}',  # Bagging
            f'{rf_imp:.4f}',   # RF
            f'{(bag_imp + rf_imp)/2:.4f}'  # Среднее
        ])
    
    # Сортируем по средней важности
    comparison_data.sort(key=lambda x: float(x[4]), reverse=True)
    
    fig_table = go.Figure(data=[go.Table(
        header=dict(values=['Ранг', 'Признак', 'Bagging', 'Random Forest', 'Среднее'],
                    fill_color='lightblue',
                    align='center',
                    font=dict(size=12, color='black')),
        cells=dict(values=list(zip(*comparison_data)),
                   fill_color='white',
                   align='center',
                   font=dict(size=11)))
    ])
    
    fig_table.update_layout(
        title='Интерактивная таблица сравнения важности признаков',
        height=400
    )
    
    fig_table.show()
    
else:
    print("Совместимая модель не найдена. Используем основные модели.")
    
    # Анализ только для Random Forest
    rf_importances = models['Random Forest'].feature_importances_
    fig_rf = create_importance_chart(models['Random Forest'], feature_names, 
                                   "Random Forest - Важность признаков")
    if fig_rf:
        fig_rf.show()

СТАТИСТИЧЕСКИЙ АНАЛИЗ ВАЖНОСТИ ПРИЗНАКОВ


Bagging Classifier:
  Среднее: 0.0333
  Медиана: 0.0030
  Стд. отклонение: 0.0758

Random Forest Classifier:
  Среднее: 0.0333
  Медиана: 0.0109
  Стд. отклонение: 0.0394

Корреляция важности между методами: 0.7509

Топ-10 признаков по средней важности:
----------------------------------------------------------------------
Ранг Признак      Bagging    RF         Среднее   
----------------------------------------------------------------------
1    Признак_23   0.2778     0.1355     0.2066    
2    Признак_28   0.3003     0.1031     0.2017    
3    Признак_24   0.1004     0.1365     0.1184    
4    Признак_21   0.1485     0.0658     0.1071    
5    Признак_8    0.0743     0.0915     0.0829    
6    Признак_7    0.0022     0.0689     0.0356    
7    Признак_3    0.0009     0.0680     0.0345    
8    Признак_1    0.0011     0.0587     0.0299    
9    Признак_27   0.0080     0.0409     0.0245    
10   Признак_14   0.0078     0.0399     0.0238    


---

## 10. Week 4: TO-DO

### Инструкции
- Выполните все задания в отдельном Jupyter Notebook
- Используйте комментарии в формате NumPy Docstring для всех функций
- Представьте результаты с помощью визуализаций Plotly
- Приложите письменные выводы и интерпретацию результатов

---

### Задание 1: Реализация модифицированного алгоритма (30%)

**Подзадание 1.1 (15%)**: Реализуйте упрощенный Decision Tree classifier с нуля, используя только NumPy. Ваша реализация должна включать:
- Функцию для вычисления Gini Impurity
- Функцию для поиска лучшего разбиения
- Рекурсивную функцию построения дерева
- Функцию предсказания

**Подзадание 1.2 (15%)**: Реализуйте простую версию Bagging, которая:
- Создает bootstrap выборки
- Обучает несколько ваших Decision Trees
- Агрегирует предсказания через голосование
- Вычисляет OOB error

**Требования:**
- Создайте класс SimpleDecisionTree с методами fit() и predict()
- Создайте класс SimpleBagging с методами fit(), predict() и oob_score()
- Все функции должны содержать подробную документацию
- Протестируйте вашу реализацию на простых данных

---

### Задание 2: Анализ нового датасета (40%)

**Подзадание 2.1 (15%)**: Загрузите датасет Wine из sklearn.datasets и проведите разведочный анализ:
- Изучите структуру данных, распределение классов
- Создайте визуализации признаков с помощью Plotly
- Проанализируйте корреляции между признаками
- Выявите потенциальные выбросы

**Подзадание 2.2 (25%)**: Примените все изученные методы (Decision Tree, Bagging, Random Forest) к датасету Wine:
- Разделите данные на train/validation/test (60%/20%/20%)
- Обучите модели с различными гиперпараметрами
- Сравните производительность с помощью различных метрик (accuracy, precision, recall, F1)
- Создайте сводную таблицу результатов
- Визуализируйте важность признаков для каждого метода

**Критерии оценки:**
- Корректность предварительной обработки данных
- Полнота анализа (минимум 4 метрики качества)
- Качество визуализаций и интерпретации
- Обоснованность выводов

---

### Задание 3: Оптимизация гиперпараметров (20%)

**Подзадание 3.1 (10%)**: Используйте GridSearchCV для поиска оптимальных гиперпараметров Random Forest:
- Определите диапазоны для поиска: n_estimators, max_depth, max_features, min_samples_split
- Используйте 5-fold cross-validation
- Найдите лучшую комбинацию параметров
- Сравните результаты до и после оптимизации

**Подзадание 3.2 (10%)**: Постройте learning curves для лучшей модели:
- Изобразите зависимость accuracy от размера обучающей выборки
- Проанализируйте наличие overfitting/underfitting
- Визуализируйте результаты с помощью Plotly
- Сделайте выводы о необходимом размере обучающей выборки

---

### Задание 4: Интерпретация и выводы (10%)

**Подзадание 4.1 (5%)**: Проанализируйте важность признаков:
- Сравните важность признаков между разными методами
- Объясните различия в ранжировании признаков
- Предложите возможные причины этих различий
- Определите наиболее информативные признаки для классификации вин

**Подзадание 4.2 (5%)**: Напишите заключение , которое должно содержать:
- Сравнение эффективности методов на датасете Wine
- Рекомендации по выбору метода в зависимости от задачи
- Ограничения проведенного анализа
- Предложения по улучшению результатов